In [ ]:
import jax.numpy as jnp
import matplotlib.pyplot as plt

def rte_spherical_symmetry(r,a,j,I0, r0):
    return (I0 + j/a + 2*j/a**2 * 1/r0) * r0/(jnp.exp(a/2 * r0)) * jnp.exp(a/2 * r)/r - j/a -2*j/a**2 * 1/r
 